In [14]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
# reset computational graph

tf.reset_default_graph()

In [3]:
file_name = 'data.bin'

with open(file_name, 'rb') as f:
    X_train_upsampled = np.load(f).astype(np.float32)
    y_train_upsampled_onehot = np.load(f).astype(np.float32)
    X_valid = np.load(f).astype(np.float32)
    y_valid_onehot = np.load(f).astype(np.float32)
    X_test = np.load(f).astype(np.float32)
    y_test_onehot = np.load(f).astype(np.float32)
    
print(X_train_upsampled.shape)
print(y_train_upsampled_onehot.shape)
print(X_valid.shape)
print(y_valid_onehot.shape)
print(X_test.shape)
print(y_test_onehot.shape)


(38076, 10, 53)
(38076, 3)
(2212, 10, 53)
(2212, 3)
(2212, 10, 53)
(2212, 3)


In [4]:
# data iteration parameters

buffer_size = 500 # buffer size for data shuffling
batch_size = 32

In [5]:
# define training dataset

dx_train = tf.data.Dataset.from_tensor_slices(X_train_upsampled)
dy_train = tf.data.Dataset.from_tensor_slices(y_train_upsampled_onehot)
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(buffer_size).repeat().batch(batch_size)

In [6]:
# define validation set

dx_valid = tf.data.Dataset.from_tensor_slices(X_valid)
dy_valid = tf.data.Dataset.from_tensor_slices(y_valid_onehot)
valid_dataset = tf.data.Dataset.zip((dx_valid, dy_valid)).shuffle(buffer_size).repeat().batch(batch_size)

In [7]:
# create general iterator

iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)

In [8]:
# make datasets that we can initialize separately, but using the same structure via the common iterator

training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

In [9]:
# neural network parameters and structure

input_dim = 53
output_dim = 3
num_steps = 10
state_size = 100
num_layers = 3
learning_rate = 1e-4
global_dropout = .8

prob = tf.placeholder_with_default(1.0, shape=())

def rnn_cell_with_dropout():
        cell = tf.nn.rnn_cell.BasicRNNCell(num_units=state_size)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=prob)
        return cell

def nn_model(in_data):  

    cells = [rnn_cell_with_dropout() for _ in range(num_layers)]
    cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=prob)
    
    init_state = cell.zero_state(batch_size, tf.float32)
    rnn_outputs, _ = tf.nn.dynamic_rnn(cell, in_data, initial_state=init_state)

    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, output_dim])
        b = tf.get_variable('b', [output_dim], initializer=tf.constant_initializer(0.0))

    # reshape rnn_outputs and y so we can get the logits in a single matmul
    rnn_outputs = rnn_outputs[:, num_steps-1, :] # keep only the last sequence
    rnn_outputs = tf.reshape(rnn_outputs, [-1, state_size])

    return tf.matmul(rnn_outputs, W) + b


In [10]:
# complete computational graph

# forward
next_element = iterator.get_next()
logits = nn_model(next_element[0])
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))

# backward
optimizer = tf.train.AdamOptimizer().minimize(loss)

# calculate accuracy
label = tf.argmax(next_element[1], 1)
prediction = tf.argmax(logits, 1)
equality = tf.equal(prediction, label)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))


In [17]:
# run the training

epochs = 100
training_rounds = int(X_train_upsampled.shape[0] / batch_size)
validation_rounds = int(X_valid.shape[0] / batch_size)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:

    sess.run(init_op)
    
    for i in range(epochs):
        
        # training
        
        sess.run(training_init_op)
        accumulated_acc = 0
        accumulated_loss = 0
        for j in range(training_rounds):
            l, _, acc = sess.run([loss, optimizer, accuracy], feed_dict={prob: global_dropout})
            accumulated_loss += l                                                             
            accumulated_acc += acc
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, accumulated_loss / training_rounds, accumulated_acc / training_rounds * 100))

        # validation
        
        sess.run(validation_init_op)
        accumulated_acc = 0
        accumulated_loss = 0
        labels = np.array([])
        predictions = np.array([])
        for j in range(validation_rounds):
            l, label_, prediction_, acc = sess.run([loss, label, prediction, accuracy], feed_dict={prob: 1.0})
            accumulated_loss += l
            accumulated_acc += acc
            labels = np.concatenate((labels, label_))
            predictions = np.concatenate((predictions, prediction_))
        print("Epoch: {}, loss: {:.3f}, validation accuracy: {:.2f}%".format(i, accumulated_loss / validation_rounds, accumulated_acc / validation_rounds * 100))
        conf = confusion_matrix(labels, predictions)
        print(conf)
#         plt.imshow(conf, cmap='binary', interpolation='None')
#         plt.show()

Epoch: 0, loss: 33.901, training accuracy: 45.28%
Epoch: 0, loss: 28.786, validation accuracy: 57.02%
[[  89  126   24]
 [ 373 1052  313]
 [  28   85  118]]
Epoch: 1, loss: 32.332, training accuracy: 48.96%
Epoch: 1, loss: 28.962, validation accuracy: 58.51%
[[  88  119   31]
 [ 345 1085  307]
 [  28   86  119]]
Epoch: 2, loss: 31.968, training accuracy: 49.80%
Epoch: 2, loss: 27.943, validation accuracy: 60.19%
[[  94  124   20]
 [ 319 1130  288]
 [  30   98  105]]
Epoch: 3, loss: 31.777, training accuracy: 50.27%
Epoch: 3, loss: 26.961, validation accuracy: 61.32%
[[  86  134   19]
 [ 313 1174  249]
 [  22  117   94]]
Epoch: 4, loss: 31.645, training accuracy: 50.71%
Epoch: 4, loss: 29.722, validation accuracy: 56.70%
[[ 104  105   30]
 [ 381 1031  324]
 [  27   89  117]]
Epoch: 5, loss: 31.560, training accuracy: 50.97%
Epoch: 5, loss: 29.604, validation accuracy: 57.52%
[[ 100  103   35]
 [ 350 1049  338]
 [  29   83  121]]
Epoch: 6, loss: 31.320, training accuracy: 51.73%
Epoch: 6

Epoch: 54, loss: 27.906, training accuracy: 59.42%
Epoch: 54, loss: 28.709, validation accuracy: 57.38%
[[  86  107   46]
 [ 390 1082  264]
 [  43   91   99]]
Epoch: 55, loss: 27.892, training accuracy: 59.48%
Epoch: 55, loss: 29.892, validation accuracy: 53.17%
[[ 95 100  42]
 [434 981 324]
 [ 51  83  98]]
Epoch: 56, loss: 27.953, training accuracy: 59.63%
Epoch: 56, loss: 29.443, validation accuracy: 53.80%
[[ 98  95  45]
 [427 995 315]
 [ 48  90  95]]
Epoch: 57, loss: 28.019, training accuracy: 59.22%
Epoch: 57, loss: 29.376, validation accuracy: 53.17%
[[ 95 104  39]
 [442 981 314]
 [ 48  87  98]]
Epoch: 58, loss: 27.902, training accuracy: 59.54%
Epoch: 58, loss: 31.212, validation accuracy: 50.05%
[[114  78  46]
 [509 891 337]
 [ 55  78 100]]
Epoch: 59, loss: 28.018, training accuracy: 59.59%
Epoch: 59, loss: 30.158, validation accuracy: 51.36%
[[ 96  97  45]
 [424 937 377]
 [ 52  79 101]]
Epoch: 60, loss: 28.125, training accuracy: 59.12%
Epoch: 60, loss: 30.254, validation accu